In [10]:
import json
import random
import requests
from tqdm.notebook import trange, tqdm

In [2]:
def loadjson(filename):
    f = open(filename, 'r')
    return json.load(f)

def makecons(keys):
    keys = keys.replace(' ', '').split(',')
    init = f"def __init__(self, {', '.join(keys)}):\n" + '\n'.join([f"    self.{key} = {key}" for key in keys])
    todict = "def todict(self):\n    return {\n" + ',\n'.join([f"        '{key}' : self.{key}" for key in keys]) + '\n    }'
    print(init)
    print()
    print(todict)
    
def getJsonFromUrl(url):
    x = requests.get(url)
    return json.loads(x.text)

def dumpfile(dic, filename):
    f = open(filename, 'w+')
    x = json.dumps(dic)
    f.write(x)

In [3]:
class Folder:
    def __init__(self, idx, title, parentFolder, openDate, closeDate):
        self.idx = idx
        self.title = title
        self.parentFolder = parentFolder
        self.openDate = openDate
        self.closeDate = closeDate
        
    def todict(self):
        return {
            "idx" : self.idx,
            "title" : self.title,
            "parentFolder" : self.parentFolder,
            "openDate" : self.openDate,
            "closeDate" : self.closeDate
        }
    
class File:
    def __init__(self, idx, title, path, size, parentFolder, openDate, closeDate):
        self.idx = idx
        self.title = title
        self.path = path
        self.size = size
        self.parentFolder = parentFolder
        self.openDate = openDate
        self.closeDate = closeDate

    def todict(self):
        return {
            'idx' : self.idx,
            'title' : self.title,
            'path' : self.path,
            'size' : self.size,
            'parentFolder' : self.parentFolder,
            'openDate' : self.openDate,
            'closeDate' : self.closeDate
        } 
    
def create_main_mod_folder(mod):
    code, year, sem = mod.split('-')
    year = int(year)
    sem = int(sem)
    year = year + 1 if sem == 2 else year
    startMonth = 8 if sem == 1 else 1
    endMonth = 12 if sem == 1 else 5
    idx = mod + '-Main'
    title = 'All Files'
    parentFolder = None
    openDate = f"{year}-{startMonth}-01T12:00:00"
    closeDate = f"{year}-{endMonth}-31T12:00:00"
    return Folder(idx, title, parentFolder, openDate, closeDate)

In [4]:
class Lesson:
    def __init__(self, code, moduleId, startTime, endTime, venue, day, weeks, lessonType):
        self.code = code
        self.moduleId = moduleId
        self.startTime = startTime
        self.endTime = endTime
        self.venue = venue
        self.day = day
        self.weeks = weeks
        self.lessonType = lessonType

    def todict(self):
        return {
            'code' : self.code,
            'moduleId' : self.moduleId,
            'startTime' : self.startTime,
            'endTime' : self.endTime,
            'venue' : self.venue,
            'day' : self.day,
            'weeks' : self.weeks,
            'lessonType' : self.lessonType
        }

In [5]:
class Module:
    def __init__(self, 
                 mod_id, 
                 title, 
                 description, 
                 credits
                ):
        self.mod_id = mod_id
        self.title = title
        self.description = description
        self.credits = credits
        self.files = create_main_mod_folder(mod_id)
    
    def todict(self):
        return {
            "id" : self.mod_id,
            "title" : self.title,
            "description" : self.description,
            "credits" : self.credits,
            "files" : self.files.todict()
        }

In [24]:
N = 40
all_modules = dict(loadjson('_AllModules2018.json'))
sample = random.sample(list(all_modules), N)

In [25]:
lessons = []
modules = []
for mod in tqdm(sample):
    for year in [2019, 2020, 2021]:
            mod_id = f"{mod}-{year}"
            title = all_modules[mod]
            mod_data = {}
            try:
                mod_data = getJsonFromUrl(f'https://api.nusmods.com/v2/{year}-{year + 1}/modules/{mod}.json')
            except:
                continue
            description = mod_data.get('description')
            credits = mod_data.get('moduleCredit')
            sem_data = mod_data.get('semesterData')
            for sem in [1, 2]:
                current_sem_mod_id = f"{mod_id}-{sem}"
                current_sem_data = list(filter(lambda x: x.get('semester') == sem, sem_data))
                if not len(current_sem_data): continue
                current_sem_lessons = current_sem_data[0].get('timetable')
                i = 0
                for lesson in current_sem_lessons:
                    code = lesson.get('classNo')
                    moduleId = current_sem_mod_id
                    startTime = lesson.get('startTime')
                    endTime = lesson.get('endTime')
                    venue = lesson.get('venue')
                    day = lesson.get('day')
                    weeks = lesson.get('weeks')
                    lessonType = lesson.get('lessonType')
                    if lessonType not in ['Lecture', 'Tutorial']: continue
                    lessonObj = Lesson(code, moduleId, startTime, endTime, venue, day, weeks, lessonType)
                    lessons.append(lessonObj)
                    i += 1
                if i == 0: continue
                modules.append(Module(current_sem_mod_id, title, description, credits))

  0%|          | 0/40 [00:00<?, ?it/s]

In [26]:
modules = [module.todict() for module in modules]
lessons = [lesson.todict() for lesson in lessons]
dumpfile(modules, 'modules.json')
dumpfile(lessons, 'lessons.json')

In [27]:
len(modules)

86

In [28]:
len(lessons)

445